In [ ]:
import pandas as pd
import os
import numpy as np
import datetime as dt
from os import listdir
from os.path import isfile, join

In [2]:
idx = pd.IndexSlice

***

# Data collection

## IT_Op_w_Products

In [3]:
op_w_prod = pd.read_csv(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Data export\IT_Op_w_Products.csv", encoding='latin-1', sep=";")

for col in op_w_prod.columns:
    if "Date" in col:
        op_w_prod[col] = op_w_prod[col].astype(str)
        op_w_prod[col] = pd.to_datetime(op_w_prod[col], format="%d/%m/%Y")
    else:
        pass


columns = [c for c in op_w_prod.columns if not "Currency" in c]
op_w_prod = op_w_prod.loc[:,columns]

num = []

for col in op_w_prod.columns:
    if "Amount" in col:
        num.append(col)
    else:
        pass
    
op_w_prod[num] = op_w_prod[num].apply(lambda x: x.str.replace(",",".", regex=False)).astype(float)

## IT_Opp_by_Created_Date

In [ ]:
op_by_date = pd.read_csv(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Data export\IT_Opp_by_Created_Date.csv", encoding='latin-1', sep=";")

op_by_date["Created Date"] = pd.to_datetime(op_by_date["Created Date"].astype(str), format="%d/%m/%Y")
op_by_date["Close Date"] = pd.to_datetime(op_by_date["Close Date"].astype(str), format="%d/%m/%Y")
op_by_date["Original Close Date"] = pd.to_datetime(op_by_date["Original Close Date"].astype(str), format="%d/%m/%Y")
op_by_date["Last Stage Change Date"] = pd.to_datetime(op_by_date["Last Stage Change Date"].astype(str), format="%d/%m/%Y")
op_by_date["Last Modified Date"] = pd.to_datetime(op_by_date["Last Modified Date"].astype(str), format="%d/%m/%Y")

columns = [c for c in op_by_date.columns if not "Currency" in c]
op_by_date = op_by_date[columns]

op_by_date.loc[:, ["Subscription Amount", "Non-Subscription Amount","Incremental Amount","weighted amount"]] = op_by_date.loc[:, ["Subscription Amount", "Non-Subscription Amount","Incremental Amount","weighted amount"]].apply(lambda x: x.str.replace(",",".", regex=False)).astype(float)

## Inflow

In [ ]:
inflow_prodotti = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\inflow_prodotti\\"

onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\inflow_prodotti") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\inflow_prodotti", f))]

df = pd.DataFrame()

for file in onlyfiles:
    data = pd.read_excel(inflow_prodotti+file)
    df = df.append(data)
    
for col in df.select_dtypes(include=[object]).columns:
    df[col] = df[col].astype(str)

## Budget

In [ ]:
budget_prodotti = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\budget_prodotti\\"

onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\budget_prodotti") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\budget_prodotti", f))]

bdg = pd.DataFrame()

for file in onlyfiles:
    data = pd.read_excel(budget_prodotti+file)
    bdg = bdg.append(data)
    
for col in bdg.select_dtypes(include=[object]).columns:
    bdg[col] = bdg[col].astype(str)

## Forecast

In [ ]:
forecast = pd.read_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\forecast_sales_director\forecast_sales_director_2022.xlsx")

## Parametriche

In [ ]:
parametriche = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\\"
onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche", f))]

par = {}
for file in onlyfiles:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+file)


In [ ]:
sales_director = par["Sales_director"][["Sales Director", "Rete", "Area territoriale"]]
linea_prodotti = par["Linea_prodotti"]

# Top five pending deals

In [4]:
opp_list = op_w_prod.groupby([op_w_prod["Close Date"].astype('datetime64[M]'), "Opportunity ID", "Opportunity Name","Stage","Ag MDM","Account Name", "Account Type","Owner Role","Product Name"]).agg({"Probability (%)":"mean", "Subscription Amount":"max", "Non-Subscription Amount":"max","Incremental Amount":"max","weighted amount":"max","Total Value":"sum","Total Price":"sum"}).sort_values(by=["Incremental Amount"], ascending=False).reset_index()

opp_list["Products"] = opp_list.groupby(["Opportunity ID", "Account Name", "Account Type", "Stage", "Close Date", "Owner Role"])["Product Name"].transform(lambda x: " | ".join(x))
top_deals = opp_list.groupby(["Opportunity ID","Opportunity Name", "Account Name", "Account Type", "Products", "Stage", "Close Date", "Owner Role"]).agg({"Incremental Amount":"max"}).reset_index().sort_values(by=["Incremental Amount"], ascending=False)

top_pend_deals = top_deals.loc[top_deals["Close Date"].dt.date >= (dt.date(2022,dt.datetime.now().month,1)), :].copy()
top_pend_deals["Account Type"].replace({"Customer":"Existing","Prospect":"New"}, inplace=True)
top_pend_deals.columns = ["Opportunity ID", "Opportunity Name","Customer", "New / Existing client", "Product", "Stage", "Status (Closed/Pipeline/Lost/Postponed)", "Owner Role", "Size (€)"]
top_pend_deals.loc[:,["Business Unit", "Comments"]] = np.NaN 

top_pend_deals_10 = top_pend_deals.iloc[0:10].copy()
top_pend_deals_10 = top_pend_deals_10[["Customer", "New / Existing client", "Business Unit", "Product", "Size (€)", "Comments", "Status (Closed/Pipeline/Lost/Postponed)"]]
top_pend_deals_10["Status (Closed/Pipeline/Lost/Postponed)"] = top_pend_deals_10["Status (Closed/Pipeline/Lost/Postponed)"].dt.strftime("Pipeline %b-%y")
top_pend_deals_10.index = np.arange(1, len(top_pend_deals_10)+1)
top_pend_deals_10.index.set_names("#", inplace=True)


In [5]:
top_pend_deals_10

,Customer,New / Existing client,Business Unit,Product,Size (€),Comments,Status (Closed/Pipeline/Lost/Postponed)
#,,,,,,,
1,SALVATORE FERRAGAMO SPA,Existing,NaN,PERSONALIZZAZIONI SOFTWARE,150000.0,NaN,Pipeline Feb-23
2,TELECOM ITALIA SPA,Existing,NaN,PROGETTO TAX PLUGIN 2.0 ATTIV.,150000.0,NaN,Pipeline Feb-23
3,Fincantieri SPA,New,NaN,PROGETTO TAX PLUGIN 2.0 ATTIV. | ANAL.CONSUL. ...,120000.0,NaN,Pipeline Mar-23
4,MEDIOBANCA SPA,Existing,NaN,PROGETTO TAX PLUGIN 2.0 ATTIV.,100000.0,NaN,Pipeline Jul-23
5,EPTA Group,New,NaN,ATL - GEST. NEWS ATTIV RIN,80000.0,NaN,Pipeline Dec-22
6,NATUZZI SPA,Existing,NaN,B.POINT SMALL - ATTIV.,60000.0,NaN,Pipeline Jul-22
7,PIAGGIO ITALIA SPA,Existing,NaN,PROGETTO TAX PLUGIN 2.0 ATTIV.,60000.0,NaN,Pipeline Nov-22
8,GENERALI ITALIA SPA,Existing,NaN,TUTTOTEL BASE 500 ANAGRAFICHE | ANAL.CONSUL. S...,50350.0,NaN,Pipeline Jan-23
9,BANCA PASSADORE & C SPA,Existing,NaN,PROGETTO TAX PLUGIN 2.0 ATTIV.,50000.0,NaN,Pipeline Nov-22


In [7]:
with pd.ExcelWriter(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Presentazioni\MDM\Top five pending deals.xlsx") as writer:
    top_pend_deals_10.to_excel(writer, sheet_name="top_pend_deals")
    opp_list.to_excel(writer, sheet_name="opp_list", index=False)

<br>

***

# Last month sales intake performance – TAA Italy

In [ ]:
columns = ["Opportunity ID","Opportunity Name", "Account Name",  "Owner Role", "Close Date","Stage", "Incremental Amount"]

filt = (op_w_prod["Stage"].str.contains("Closed Won")) & (op_w_prod["Close Date"].dt.month==(dt.datetime.now().month-1)) & (op_w_prod["Close Date"].dt.year==(dt.datetime.now().year))
closed_deals = op_w_prod.loc[filt,columns].drop_duplicates(subset=["Opportunity ID"]).sort_values(by="Incremental Amount", ascending=False).head(10)

filt = (op_w_prod["Stage"].str.contains("Closed Lost")) & (op_w_prod["Close Date"].dt.month==(dt.datetime.now().month-1)) & (op_w_prod["Close Date"].dt.year==(dt.datetime.now().year))
lost_deals = op_w_prod.loc[filt,columns].drop_duplicates(subset=["Opportunity ID"]).sort_values(by="Incremental Amount", ascending=False).head(10)

In [ ]:
closed_deals

In [ ]:
with pd.ExcelWriter(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Presentazioni\MDM\Last month sales intake performance.xlsx") as writer:
    closed_deals.to_excel(writer, sheet_name="closed_deals", index=False)
    lost_deals.to_excel(writer, sheet_name="lost_deals", index=False)

### Success

In [ ]:
columns = ["Opportunity ID","Opportunity Name", "Product Name", "Total Price", "Account Name",  "Owner Role", "Close Date","Stage", "Incremental Amount"]

filt = (op_w_prod["Stage"].str.contains("Closed Won")) & (op_w_prod["Close Date"].dt.month==(dt.datetime.now().month-1)) & (op_w_prod["Close Date"].dt.year==(dt.datetime.now().year))

closed_deals_con_prod = op_w_prod.loc[filt,:].groupby(["Opportunity Name","Account Name",  "Owner Role", "Close Date","Product Name"]).agg({"Total Price":"sum","Incremental Amount":"sum"}).sort_values(by=["Incremental Amount"], ascending=False)

In [ ]:
closed_deals_con_prod.style.format("{:,.0f}€").background_gradient().to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Presentazioni\MDM\success.xlsx")

### Lost

In [ ]:
columns = ["Opportunity ID","Opportunity Name", "Product Name", "Total Price", "Account Name",  "Owner Role", "Close Date","Stage", "Incremental Amount"]

filt = (op_w_prod["Stage"].str.contains("Closed Lost")) & (op_w_prod["Close Date"].dt.month==(dt.datetime.now().month-1)) & (op_w_prod["Close Date"].dt.year==(dt.datetime.now().year))

closed_lost_deals_con_prod = op_w_prod.loc[filt,:].groupby(["Opportunity Name","Account Name",  "Owner Role", "Close Date","Product Name"]).agg({"Total Price":"sum","Incremental Amount":"sum"}).sort_values(by=["Incremental Amount"], ascending=False)

In [ ]:
closed_lost_deals_con_prod

# Pipeline – TAA Italy

## Pipeline

In [ ]:
pipe = op_w_prod.drop_duplicates(subset=["Opportunity ID"])

### Total Pipeline

In [ ]:
filt = (pipe["Close Date"].dt.month > (dt.datetime.now().month-1)) & (~pipe["Stage"].str.contains("Closed"))
tot_pipeline = pipe.loc[filt,:].groupby([pipe["Close Date"].astype('datetime64[M]')])[["Incremental Amount"]].sum().reset_index().pivot_table(columns="Close Date", values="Incremental Amount")

In [ ]:
tot_pipeline

### Forecast 50<70 e Forecast >70

In [ ]:
filt = (pipe["Close Date"].dt.month > (dt.datetime.now().month-1)) & (pipe["Ag MDM"].str.contains("70<90|50<70"))
pipe_50_90 = pipe.loc[filt,:].groupby([pipe["Close Date"].astype('datetime64[M]'), "Stage", "Ag MDM"])[[ "weighted amount"]].sum().unstack(0).fillna(0)

In [ ]:
pipe_50_90

### Actual

In [ ]:
esc_fatture = par["Esclusione_fatture"]
esc_fatture["Numero Fattura"] = esc_fatture["Numero Fattura"].astype(str)

esc_ordini = par["Esclusione_ordini"]
esc_ordini["Numero Ordine"] = esc_ordini["Numero Ordine"].astype(str)

li_prodotti = par["Linea_prodotti"]

df_merge = df.merge(esc_fatture, on="Numero Fattura", how="left").merge(esc_ordini, on="Numero Ordine", how="left")

current_month = dt.date.today().replace(day=1)
filt = (df_merge["Data Ins. Ordine (monitoraggio)"].dt.date >= current_month) & (df_merge["Sales Director"].isin(["LARGE ACCOUNT","NORDEST IBISOFT","WKIC CENTRO SUD","WKIC NORD EST","WKIC NORD OVEST","ENTERPRISE"]))&(df_merge["Escludi fatture"].isna())&(df_merge["Escludi ordine"].isna())

In [ ]:
actual = df_merge.loc[filt,:].groupby([df_merge["Data Ins. Ordine (monitoraggio)"].astype("datetime64[M]")]).agg({"Raccolto":"sum"}).transpose()

In [ ]:
actual

### Budget

In [ ]:
bdg["Sales Director"] = bdg["Sales Director"].str.lower()
budget = bdg.merge(sales_director, on="Sales Director", how="left").merge(linea_prodotti, on= "Linea Prodotti", how="left")
budget.rename(columns={"Data Ins. Ordine (monitoraggio)":"Data"}, inplace=True)

In [ ]:
bdg_mid_month_forecast = budget.groupby(["Rete", "Linea", budget["Data"].astype('datetime64[M]')])["Budget"].sum().unstack(level=2).loc["Diretta"]
bdg_mid_month_forecast

In [ ]:
tgt = budget.groupby(["Rete", budget["Data"].astype('datetime64[M]')])["Budget"].sum().unstack(level=1)
tgt

### Forecast

In [ ]:
sales = par["Sales_director"]
sales = sales.loc[:,["Sales Director", "Area territoriale", "Rete"]]
sales["Sales Director"] = sales["Sales Director"].apply(lambda x: x.lower())

forecast["Sales director"] = forecast["Sales director"].str.lower()
fc = forecast.merge(sales, left_on= "Sales director", right_on="Sales Director", how="left").merge(linea_prodotti, on= "Linea Prodotti", how="left")
fc = fc.loc[fc["Rete"] == "Diretta", ["Giorno","Linea", "Forecast"]].copy()
fc_month_forecast = fc.groupby(["Giorno", "Linea"]).agg({"Forecast":"sum"}).unstack(0)
fc_month_forecast

### Total Pipeline FY

In [ ]:
filt = (pipe["Close Date"].dt.month > (dt.datetime.now().month-1)) & (~pipe["Stage"].str.contains("Closed"))
tot_pipeline_fy = pipe.loc[filt,"Incremental Amount"].sum()
tot_pipeline_fy

## Export to excel

In [ ]:
pipe_50_90_drop = pipe_50_90.droplevel(level=0, axis=1)
pipe_50_90_drop = pipe_50_90_drop.droplevel(level=0, axis=0)

In [ ]:
pipeline_taa_italy = pd.concat([tot_pipeline, pipe_50_90_drop,  bdg_mid_month_forecast])

In [ ]:
pipeline_taa_italy

In [ ]:
pipeline_taa_italy.iloc[0:5,6:9]

In [ ]:
pipeline_taa_italy.iloc[0:5,6:9].to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Presentazioni\MDM\Pipeline.xlsx")

# Sales KPIs

In [ ]:
filt_rows = (df["Data Ins. Ordine (monitoraggio)"].dt.year == 2022)&(df["Raccolto"].notna())
columns = ["Cliente Merce", "Sales Director", "Canale di VENDITA", "Linea Prodotti", "Numero Ordine", "Numero Fattura", "Data Ins. Ordine (monitoraggio)", "Raccolto"]

df_2022 = df.loc[filt_rows, columns].reset_index(drop=True)
df_2022["Sales Director"] = df_2022["Sales Director"].str.lower()

esc_fatture = par["Esclusione_fatture"]
esc_fatture["Numero Fattura"] = esc_fatture["Numero Fattura"].astype(str)

esc_ordini = par["Esclusione_ordini"]
esc_ordini["Numero Ordine"] = esc_ordini["Numero Ordine"].astype(str)

sales_director = par["Sales_director"]
sales_director["Sales Director"] = sales_director.loc[:,"Sales Director"].str.lower()
sales_director = sales_director[["Sales Director", "Rete", "Rete2", "Area territoriale"]]

li_prodotti = par["Linea_prodotti"]

inflow = df_2022.merge(esc_fatture, on="Numero Fattura", how="left").merge(esc_ordini, on="Numero Ordine", how="left").merge(li_prodotti, on="Linea Prodotti", how="left").merge(sales_director, on="Sales Director", how="left")

esclusioni = (inflow["Canale di VENDITA"] != "E-Commerce")&(inflow["Escludi fatture"].isna())&(inflow["Escludi ordine"].isna())

inflow = inflow.loc[esclusioni, ['Data Ins. Ordine (monitoraggio)', "Cliente Merce", "Numero Ordine", "Rete", "Rete2","Area territoriale", 'Sales Director', 'Linea', 'Raccolto']].reset_index(drop=True)

inflow.rename(columns={"Data Ins. Ordine (monitoraggio)":'Giorno'}, inplace=True)

#### Ordine medio per cliente  
Consideriamo l'ordine medio per cliente, anche se nella slide c'è scritto "Average deal size"

In [ ]:
inflow["N. Clienti"] = inflow.groupby(["Rete2", inflow["Giorno"].astype('datetime64[M]')], dropna=False)["Cliente Merce"].transform("nunique")
inflow_gr = inflow.groupby(["Rete2", inflow["Giorno"].astype('datetime64[M]')], dropna=False).agg({"Raccolto":"sum", "N. Clienti":"max"})

inflow_gr["Avg deal size"] = inflow_gr["Raccolto"] / inflow_gr["N. Clienti"]

inflow_gr["Avg deal size"].unstack(level=1).style.format("{:,.0f}").background_gradient()

#### Ordine medio per deal

In [ ]:
inflow["N. Ordini"] = inflow.groupby(["Rete2", inflow["Giorno"].astype('datetime64[M]')], dropna=False)["Numero Ordine"].transform("nunique")
inflow_gr = inflow.groupby(["Rete2", inflow["Giorno"].astype('datetime64[M]')], dropna=False).agg({"Raccolto":"sum", "N. Ordini":"max"})

inflow_gr["Avg deal size"] = inflow_gr["Raccolto"] / inflow_gr["N. Ordini"]

inflow_gr["Avg deal size"].unstack(level=1).style.format("{:,.0f}").background_gradient()

# Sales Quota by Sales Rep and Sales Team (direct channel only)

In [ ]:
periodo = (df["Data Ins. Ordine (monitoraggio)"].dt.year == 2022) & (df["Data Ins. Ordine (monitoraggio)"].dt.quarter.isin([1,2]))
filt_rows = (periodo)

columns = ["Cliente Merce", "Sales Director", "Agente dell'ORDINE", "Canale di VENDITA", "Linea Prodotti", "Numero Ordine", "Numero Fattura", "Data Ins. Ordine (monitoraggio)", "Raccolto", "Budget  assegnato"]

df2 = df.loc[filt_rows, columns].reset_index(drop=True)
df2["Sales Director"] = df2["Sales Director"].str.lower()

esc_fatture = par["Esclusione_fatture"]
esc_fatture["Numero Fattura"] = esc_fatture["Numero Fattura"].astype(str)

esc_ordini = par["Esclusione_ordini"]
esc_ordini["Numero Ordine"] = esc_ordini["Numero Ordine"].astype(str)

sales_director = par["Sales_director"]
sales_director["Sales Director"] = sales_director.loc[:,"Sales Director"].str.lower()
sales_director = sales_director[["Sales Director", "Rete"]]

li_prodotti = par["Linea_prodotti"]

agente_ordine = par["Agente_ordine"]
agente_ordine.rename(columns = {"Agente dell'ORDINE":"Sales rep"}, inplace=True) 

sales_quota = df2.merge(esc_fatture, on="Numero Fattura", how="left").merge(esc_ordini, on="Numero Ordine", how="left").merge(li_prodotti, on="Linea Prodotti", how="left").merge(sales_director, on="Sales Director", how="left").merge(agente_ordine, left_on="Agente dell'ORDINE", right_on="Codice agente", how="left")

esclusioni = (sales_quota["Canale di VENDITA"] != "E-Commerce") & (sales_quota["Escludi fatture"].isna()) & (sales_quota["Escludi ordine"].isna()) & (sales_quota["Rete"] == "Diretta") & (sales_quota["Sales Director"] != "large account")

sales_quota = sales_quota.loc[esclusioni, ['Data Ins. Ordine (monitoraggio)', "Codice agente", "Sales rep", 'Raccolto', "Budget  assegnato"]].reset_index(drop=True)

sales_quota.rename(columns={"Data Ins. Ordine (monitoraggio)":'Giorno'}, inplace=True)

In [ ]:
sales_quota_gr = sales_quota.groupby(["Sales rep"]).agg({"Raccolto":"sum", "Budget  assegnato":"sum"}).reset_index().sort_values(by=["Budget  assegnato"], ascending=False)

In [ ]:
sales_quota_gr

# Sales Activity

In [ ]:
sales_activity = op_w_prod.loc[(op_w_prod["Created Date"].dt.date >=dt.date(2022,1,1)) & (op_w_prod["Created Date"].dt.quarter.isin([1,2,3,4])),:].copy()
sales_activity["Created Week"] = op_w_prod["Created Date"].dt.isocalendar().week
sales_activity["Created Quarter"] = op_w_prod["Created Date"].dt.quarter

In [ ]:
sales_activity = sales_activity.groupby(["Created Week", "Created Quarter"]).agg({"Opportunity ID":"nunique"}).reset_index()

In [ ]:
with pd.ExcelWriter(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Presentazioni\MDM\Sales Quota & Sales Activity.xlsx", engine="openpyxl") as writer:
    sales_quota_gr.to_excel(writer, sheet_name="sales_quota", index=False)
    sales_activity.to_excel(writer, sheet_name="op_by_week", index=True)

# Clienti SMART Skills

In [ ]:
inflow_prodotti = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\inflow_prodotti\\"

onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\inflow_prodotti") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\inflow_prodotti", f))]

df = pd.DataFrame()

for file in onlyfiles:
    data = pd.read_excel(inflow_prodotti+file)
    df = df.append(data)

In [ ]:
for col in df.select_dtypes(include=[object]).columns:
    df[col] = df[col].astype(str)

In [ ]:
parametriche = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\\"
onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche", f))]

par = {}
for file in onlyfiles:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+file)

In [ ]:
par.keys()

In [ ]:
columns = ["Cliente Merce", "Sales Director", "Canale di VENDITA", "Linea Prodotti", "Codice PRODOTTO", "Numero Ordine", "Numero Fattura", "Data Ins. Ordine (monitoraggio)", "Raccolto"]

df_smart = df.loc[:, columns].reset_index(drop=True)

df_smart["Sales Director"] = df_smart["Sales Director"].str.lower()
df_smart["Codice PRODOTTO"] = df_smart["Codice PRODOTTO"].astype(str)

esc_fatture = par["Esclusione_fatture"]
esc_fatture["Numero Fattura"] = esc_fatture["Numero Fattura"].astype(str)

esc_ordini = par["Esclusione_ordini"]
esc_ordini["Numero Ordine"] = esc_ordini["Numero Ordine"].astype(str)

sales_director = par["Sales_director"]
sales_director["Sales Director"] = sales_director.loc[:,"Sales Director"].str.lower()
sales_director = sales_director[["Sales Director", "Rete", "Rete3", "Area territoriale"]]

prodotti = par["Classificazione_codice_prodotto"]
prodotti["Codice PRODOTTO"] = prodotti["Codice PRODOTTO"].astype(str)

li_prodotti = par["Linea_prodotti"]

clienti_smart = df_smart.merge(esc_fatture, on="Numero Fattura", how="left").merge(esc_ordini, on="Numero Ordine", how="left").merge(li_prodotti, on="Linea Prodotti", how="left").merge(sales_director, on="Sales Director", how="left").merge(prodotti, on="Codice PRODOTTO", how="left")

esclusioni = (clienti_smart["Escludi fatture"].isna())&(clienti_smart["Escludi ordine"].isna())&(clienti_smart["Sub solution"] == "Smart - Skills")

clienti_smart = clienti_smart.loc[esclusioni, ['Data Ins. Ordine (monitoraggio)', "Cliente Merce", "Numero Ordine", "Rete3", "Canale di VENDITA",'Linea', 'Raccolto']].reset_index(drop=True)

clienti_smart.rename(columns={"Data Ins. Ordine (monitoraggio)":'Giorno'}, inplace=True)

## Numero clienti

In [ ]:
arr = clienti_smart["Giorno"].astype('datetime64[M]').sort_values().unique()
d_rete = {}
d_canale = {}

for i,elem in enumerate(arr):
    temp = clienti_smart.loc[(clienti_smart["Giorno"].astype('datetime64[M]') >= "2021-04-01") & 
                             (clienti_smart["Giorno"].astype('datetime64[M]') <= elem) & 
                             (clienti_smart["Canale di VENDITA"] != "E-Commerce"), :].groupby(["Rete3"]).agg({"Cliente Merce":"nunique"}).reset_index()
    d_rete[f"d_{i}"] = temp["Cliente Merce"].to_numpy()
    
    temp = clienti_smart.loc[(clienti_smart["Giorno"].astype('datetime64[M]') >= "2021-04-01") & 
                             (clienti_smart["Giorno"].astype('datetime64[M]') <= elem) & 
                             (clienti_smart["Canale di VENDITA"] == "E-Commerce"), :].groupby(["Canale di VENDITA"]).agg({"Cliente Merce":"nunique"}).reset_index()
    d_canale[f"d_{i}"] = temp["Cliente Merce"].to_numpy()

df_rete = pd.DataFrame.from_dict(data= d_rete.items())
df_rete = pd.DataFrame(df_rete[1].tolist())
df_rete = df_rete.pivot_table(columns=df_rete.index)

df_canale = pd.DataFrame.from_dict(data= d_canale.items())
df_canale = pd.DataFrame(df_canale[1].tolist())
df_canale = df_canale.pivot_table(columns=df_canale.index)

clienti_unici = pd.concat([df_rete, df_canale], axis=0)
clienti_unici.columns = arr
clienti_unici.index = ["Diretta", "Indiretta", "E-commerce"]

In [ ]:
clienti_unici = clienti_unici.transpose()
clienti_unici["Traditional channels"] = clienti_unici["Diretta"] + clienti_unici["Indiretta"]

In [ ]:
clienti_unici.loc["2022-01-01":, ["Traditional channels", "E-commerce"]]

In [ ]:
inflow_progr_Anna = clienti_smart.loc[(clienti_smart["Canale di VENDITA"] != "E-Commerce") & (clienti_smart["Giorno"] >= "2022-01-01"), :].groupby([clienti_smart["Giorno"].astype("datetime64[M]")])["Raccolto"].sum().reset_index()
inflow_progr_Anna["Anna"] = inflow_progr_Anna["Raccolto"].cumsum()

inflow_progr_Ben = clienti_smart.loc[(clienti_smart["Canale di VENDITA"] == "E-Commerce") & (clienti_smart["Giorno"] >= "2022-01-01"), :].groupby([clienti_smart["Giorno"].astype("datetime64[M]")])["Raccolto"].sum().reset_index()
inflow_progr_Ben["Ben"] = inflow_progr_Ben["Raccolto"].cumsum()

inflow_progr = inflow_progr_Anna.merge(inflow_progr_Ben, how="outer", on="Giorno")
inflow_progr = inflow_progr.loc[:,["Giorno","Anna","Ben"]]

In [ ]:
inflow_progr

## Numero ordini

In [ ]:
arr = ["2021-04-01","2021-05-01","2021-06-01","2021-07-01","2021-08-01","2021-09-01","2021-10-01","2021-11-01","2021-12-01","2022-01-01","2022-02-01","2022-03-01","2022-04-01","2022-05-01"]
d_rete = {}
d_canale = {}

for i,elem in enumerate(arr):
    temp = clienti_smart.loc[(clienti_smart["Giorno"].astype('datetime64[M]') >= "2021-04-01") & 
                             (clienti_smart["Giorno"].astype('datetime64[M]') <= elem) & 
                             (clienti_smart["Canale di VENDITA"] != "E-Commerce"), :].groupby(["Rete3"]).agg({"Numero Ordine":"nunique"}).reset_index()
    d_rete[f"d_{i}"] = temp["Numero Ordine"].to_numpy()
    
    temp = clienti_smart.loc[(clienti_smart["Giorno"].astype('datetime64[M]') >= "2021-04-01") & 
                             (clienti_smart["Giorno"].astype('datetime64[M]') <= elem) & 
                             (clienti_smart["Canale di VENDITA"] == "E-Commerce"), :].groupby(["Canale di VENDITA"]).agg({"Numero Ordine":"nunique"}).reset_index()
    d_canale[f"d_{i}"] = temp["Numero Ordine"].to_numpy()

df_rete = pd.DataFrame.from_dict(data= d_rete.items())
df_rete = pd.DataFrame(df_rete[1].tolist())
df_rete = df_rete.pivot_table(columns=df_rete.index)

df_canale = pd.DataFrame.from_dict(data= d_canale.items())
df_canale = pd.DataFrame(df_canale[1].tolist())
df_canale = df_canale.pivot_table(columns=df_canale.index)

ordini_unici = pd.concat([df_rete, df_canale], axis=0)
ordini_unici.columns = arr
ordini_unici.index = ["Diretta", "Indiretta", "E-commerce"]

In [ ]:
ordini_unici

In [ ]:
with pd.ExcelWriter(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Presentazioni\MDM\MDM - SMART skills2.xlsx", engine="openpyxl") as writer:
    clienti_unici.to_excel(writer, sheet_name="clienti_unici", index=False)
    ordini_unici.to_excel(writer, sheet_name="ordini_unici", index=False)